In [1]:
import torch
import os

In [2]:
import glob
len(glob.glob("/opt/ml/input/data/ICDAR17_TEST/images/*.*"))

2970

In [3]:
import json

In [4]:
with open('/opt/ml/input/data/AIHUB_Korean/aihub_gt/gt_aihub_1098.json') as test:
    x = json.load(test)
print(x)

{'annotations': [{'bbox': [121, 637, 1352, 183], 'id': 1, 'text': '축산물직거래장터', 'image_id': 1}, {'bbox': [378, 828, 402, 67], 'id': 2, 'text': 'xxx', 'image_id': 1}, {'bbox': [320, 0, 185, 111], 'id': 3, 'text': 'xxx', 'image_id': 1}, {'bbox': [246, 512, 309, 63], 'id': 4, 'text': 'xxx', 'image_id': 1}, {'bbox': [0, 946, 1062, 254], 'id': 5, 'text': 'xxx', 'image_id': 1}], 'cropLabels': [], 'images': [{'file_name': '간판_가로형간판_001098.jpg', 'date_created': '20210204', 'width': 1600, 'id': 1, 'height': 1200}], 'info': {'date_created': '20210204', 'name': '간판_가로형간판_001098', 'description': ''}, 'metadata': [{'area': '전라권', 'wordfont': '활자체', 'outline': '상', 'illuminance': '밝음', 'wordsize': '소', 'light': '자연광', 'wordconnection': '빈공간없이 이어짐', 'subclass': '가로형간판', 'weather': '맑음', 'device': '스마트폰', 'wordcolor': '여러색', 'class': '실외간판', 'wordorientation': '가로'}]}


In [5]:
box = x['annotations'][0]['text']
box

'축산물직거래장터'

In [6]:
if data["metadata"][0]["wordorientation"] == '가로':
    orientation = "Horizontal"
elif data["metadata"][0]["wordorientation"] == '세로':
    orientation = "Vertical"
else:
    orientation = "Irregular"
orientation

NameError: name 'data' is not defined

In [7]:
file_json = glob.glob('/opt/ml/input/data/AIHUB_Korean/aihub_gt/*.json')
print(len(file_json))
images = dict()
result = dict()
for file in file_json:
    with open(file) as f:
        data = json.load(f)
    result['images'] = dict()
    fname = '_'.join(file.split('/')[-1].split('_')[1:]).split('.')[0]+'.jpg'
    images[fname]=dict()
    images[fname]['img_h'] = data['images'][0]['height']
    images[fname]['img_w'] = data['images'][0]['width']
    images[fname]['words'] = dict()
    
    words = images[fname]['words']

    anno_len = len(data['annotations'])
    for idx in range(anno_len):
        words[str(idx)]=dict()
        words[str(idx)]["points"] = list()
        bbox = data['annotations'][idx]['bbox']

        points = words[str(idx)]["points"]      #rect
        points.append(bbox[:2])   #x1,y1
        points.append([(bbox[0]+bbox[2]), bbox[1]])
        points.append([(bbox[0]+bbox[2]), bbox[1]+bbox[3]])
        points.append([bbox[0], bbox[1]+bbox[3]])

        if data['annotations'][idx]['text']=='xxx':
            words[str(idx)]['transcription']="###"
            words[str(idx)]['language']=["en"]
            words[str(idx)]['illegibility']=data['annotations'][idx]['text']=='xxx'
        else:
            words[str(idx)]['transcription']=data['annotations'][idx]['text']
            words[str(idx)]['language']=["ko"]
            words[str(idx)]['illegibility']=data['annotations'][idx]['text']=='xxx'

        if data["metadata"][0]["wordorientation"] == '가로':
            words[str(idx)]['orientation'] = "Horizontal"
        elif data["metadata"][0]["wordorientation"] == '세로':
            words[str(idx)]['orientation'] = "Vertical"
        else:
            words[str(idx)]['orientation'] = "Irregular" 
        words[str(idx)]['word_tags'] = None   

    images[fname]['tags'] = None
    images[fname]['license_tag']=dict()
    license = images[fname]['license_tag']
    license['usability'] = True
    license['public'] = True
    license['commercial'] = True
    license['type'] = "CC-BY-SA"
    license['holder'] = None
    
result['images']=images

result

1140


{'images': {'aihub_14.jpg': {'img_h': 1200,
   'img_w': 1600,
   'words': {'0': {'points': [[151, 459],
      [1398, 459],
      [1398, 840],
      [151, 840]],
     'transcription': '전당포',
     'language': ['ko'],
     'illegibility': False,
     'orientation': 'Horizontal',
     'word_tags': None},
    '1': {'points': [[107, 305], [1416, 305], [1416, 462], [107, 462]],
     'transcription': '###',
     'language': ['en'],
     'illegibility': True,
     'orientation': 'Horizontal',
     'word_tags': None}},
   'tags': None,
   'license_tag': {'usability': True,
    'public': True,
    'commercial': True,
    'type': 'CC-BY-SA',
    'holder': None}},
  'aihub_572.jpg': {'img_h': 1200,
   'img_w': 1600,
   'words': {'0': {'points': [[326, 467],
      [1375, 467],
      [1375, 751],
      [326, 751]],
     'transcription': '카페달리다',
     'language': ['ko'],
     'illegibility': False,
     'orientation': 'Horizontal',
     'word_tags': None},
    '1': {'points': [[1198, 727], [1569, 727]

In [28]:
with open(os.path.join('/opt/ml/input/data/AIHUB_Korean/ufo', 'train.json'), 'w') as f:
    json.dump(result, f, indent=4)

### 이미지 복사

In [24]:
from PIL import Image
from tqdm import tqdm
image_path = glob.glob('/opt/ml/input/data/AIHUB_Korean/aihub_images/*.*')
for image_pth in tqdm(image_path):
    image = Image.open(image_pth)
    image.save(os.path.join('/opt/ml/input/data/combine_data/image',os.path.basename(image_pth)))

100%|██████████| 1140/1140 [02:09<00:00,  8.83it/s]


In [26]:
image_path = glob.glob('/opt/ml/input/data/ICDAR17_TEST/images/*.*')
for image_pth in tqdm(image_path):
    image = Image.open(image_pth)
    image.save(os.path.join('/opt/ml/input/data/combine_data/image',os.path.basename(image_pth)))

100%|██████████| 2970/2970 [11:59<00:00,  4.13it/s]


### 확장자명 JPG -> jpg

In [43]:
image_path = glob.glob('/opt/ml/input/data/Combine_Data/images/*.*')
for image_pth in image_path:
    # print(image_pth.split('.')[0])
    if image_pth.split('.')[-1] == 'JPG':
        os.rename(image_pth, image_pth.split('.')[0]+'.jpg')